This notebook provides code for downloading additional data form MODIS, Landsat and Sentinel-2, using Google Earth Engine Python API. 

# Preliminaries

Firstly, import libraries and read data provided by the competition.

In [1]:
import os, datetime
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd
import geopandas as gpd

from src import read_data, process_data, split_df

In [2]:
data = read_data('data/train_dataset_train_2.csv')
data_test = read_data('data/test_dataset_test_2.csv')
data_cat = pd.concat((data, data_test), axis=0)

Authenticate Google Earth Engine. If you are registered there, it will provide an API key to authenticate this notebook.

In [3]:
import ee
import eeconvert

In [5]:
ee.Authenticate()
ee.Initialize()

Now we define the primary funciton that will fetch time series data from Google Earth Engine.

In [16]:
def get_series(collection, features, band='ndvi', scale=30, drive_folder=None):
    """https://stackoverflow.com/questions/47633088/get-results-in-an-earth-engine-python-script"""
    def GetSeries(feature):
        def NDVIcalc(img):
            return feature.set(
                img.reduceRegion(ee.Reducer.median(), feature.geometry(), scale)
            ).set('date', img.date().format('YYYY-MM-dd'))

        series = collection.map(NDVIcalc)

        lst = series.reduceColumns(
            ee.Reducer.toList(2), ['date', band.upper()]).get('list')
        return feature.set(ee.Dictionary(ee.List(lst).flatten()))

    features_mapped = features.map(GetSeries)
    if drive_folder is None:
        result = features_mapped.getInfo()
        return pd.DataFrame([i['properties'] for i in result['features']])
    else:
        task = ee.batch.Export.table.toDrive(
            features_mapped, folder=drive_folder, fileFormat='csv')
        task.start()

# Donwnloading data

## Landsat-8

First, split data into tiles, corresponding to Landsat scenes. This is done beacause otherwise each API call would be too large and get timed out by the server.

In [7]:
# grid source: https://mgimond.github.io/ArcGIS_tutorials/Download_landsat.htm
grid = gpd.read_file('data/grid/wrs2.shp')
data_grid = data_cat.sjoin(grid[['PATH', 'ROW', 'geometry']], how='left')
gb = data_grid.groupby(['PATH', 'ROW'])
tiles = [gb.get_group(x) for x in gb.groups]

Donwnload time series directly with the funciton defined earlier.

In [8]:
# loop over tiles to get series df for them from GEE
out = []
for tile in tqdm(tiles):
    # convert gpd geometry to ee.FeatureCollection
    features = eeconvert.gdfToFc(tile.to_crs(4326)[['id', 'geometry']])
    # define ee.ImageCollection of desired sattelite
    collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_8DAY_NDVI') \
        .filterBounds(features) \
        .filterDate('2021-04-15', '2021-09-01') \
        .select('NDVI')
    series = get_series(collection, features, scale=30)
    out.append(series)
data_landsat = pd.concat(out)

  0%|          | 0/102 [00:00<?, ?it/s]

Group results by id and select maximum values in case of duplicates (scene overlapping). We choose max is better bacause in one of the images there can be cluods, so max value (possibly without coluds) is more informative.

In [ ]:
data_landsat = data_landsat.groupby('id').max().reset_index(drop=True)
data_landsat = data_landsat.loc[data_cat['id'], sorted(data_landsat.columns)]

Save the data.

In [11]:
data_landsat.iloc[:data.shape[0],:].to_csv('data/train_dataset_landsat.csv', index=False)
data_landsat.iloc[data.shape[0]:,:].to_csv('data/test_dataset_landsat.csv', index=False)

## MODIS

First, split data into tiles, corresponding to MODIS scenes.

In [4]:
# grid source: https://mgimond.github.io/ArcGIS_tutorials/Download_landsat.htm
grid = gpd.read_file('data/grid/modis_sinusoidal_grid_world.shp')
data_grid = data_cat.to_crs(grid.crs).sjoin(grid[['h', 'v', 'geometry']], how='left')
gb = data_grid.groupby(['h', 'v'])
tiles = [gb.get_group(x) for x in gb.groups]

Donwnload the data.

In [16]:
# split tiles further into even smaller dataframes of size <= 100
tiles_gen = (t for tile in tiles for t in split_df(tile, 100))

# loop over tiles to get series df for them from GEE
out = []
for tile in tqdm(tiles_gen):
    # convert gpd geometry to ee.FeatureCollection
    features = eeconvert.gdfToFc(tile.to_crs(4326)[['id', 'geometry']])
    # define ee.ImageCollection of desired sattelite
    collection = ee.ImageCollection('MODIS/MOD09GA_006_NDVI') \
        .filterBounds(features) \
        .filterDate('2021-04-15', '2021-09-01') \
        .select('NDVI')
    series = get_series(collection, features, scale=250)
    out.append(series)
data_modis = pd.concat(out)

  0%|          | 0/73 [00:00<?, ?it/s]

Get rid of duplicates and sort date columns.

In [18]:
data_modis = data_modis.groupby('id').max().reset_index(drop=True).fillna(0)
data_modis = data_modis.loc[data_cat['id'], sorted(data_modis.columns)]

Save the data to local folder.

In [22]:
data_modis.iloc[:data.shape[0],:].to_csv('data/train_dataset_modis.csv', index=False)
data_modis.iloc[data.shape[0]:,:].to_csv('data/test_dataset_modis.csv', index=False)

## Sentinel-2

In [8]:
def set_sentinel_ndvi(image):
    red = image.select('B4')
    nir = image.select('B8')
    ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
    return image.addBands(ndvi)

def set_sentinel_evi(image):
    evi = image.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', 
        {'NIR': image.select('B8').divide(10000),
         'RED': image.select('B4').divide(10000),
         'BLUE': image.select('B2').divide(10000)}
    ).rename('EVI')
    return image.addBands(evi)

First, split data into tiles, corresponding to Sentinel-2 scenes. This is done beacause otherwise each API call would be too large and get timed out by the server.

In [9]:
# grid source: https://github.com/justinelliotmeyers/Sentinel-2-Shapefile-Index
grid = gpd.read_file('data/grid/sentinel_2_index_shapefile.shp')
data_grid = data_cat.to_crs(grid.crs).sjoin(grid[['Name', 'geometry']], how='left')
gb = data_grid.groupby('Name')
tiles = [gb.get_group(x) for x in gb.groups]

For Sentinel-2, save results into Google Drive instead of downloading them directly.
Because Sentinel data is too ehavy, direct download would stall out.

In [18]:
# split tiles further into even smaller dataframes of size <= 100
tiles_gen = (t for tile in tiles for t in split_df(tile, 100))

# loop over tiles to get series df for them from GEE
for tile in tqdm(tiles_gen):
    # convert gpd geometry to ee.FeatureCollection
    features = eeconvert.gdfToFc(tile.to_crs(4326)[['id', 'geometry']])
    # define ee.ImageCollection of desired sattelite
    collection = ee.ImageCollection('COPERNICUS/S2') \
        .map(set_sentinel_ndvi) \
        .filterBounds(features) \
        .filterDate('2021-04-15', '2021-09-01') \
        .select('NDVI')
    # save results into Google Drive
    get_series(collection, features, scale=10, drive_folder='sentinel')

  0%|          | 0/187 [00:00<?, ?it/s]

Processing and donwnloading to Google Drive can be monitored here:
https://code.earthengine.google.com/tasks

After the process is finished, download the folder from Google Drive manually. 

In [283]:
dir_path = 'sentinel'
out = []
for file_name in os.listdir(dir_path):
    out.append(pd.read_csv(dir_path+'/'+file_name))
data_sentinel = pd.concat(out)

Sentinel satellite passes through our regin in midnight, thus producing data with different dates.
Fix this by mapping everything to the date before midnight.

In [294]:
data_prev = data_sentinel.iloc[:,1:56]
data_prev.columns = pd.to_datetime(data_prev.columns)
data_next = data_sentinel.iloc[:,59:]
data_next.columns = pd.to_datetime(data_next.columns) - pd.Timedelta(days=1)
data_prev.update(data_next)
data_prev['id'] = data_sentinel['id']

Group results by id and select maximum values in case of duplicates (scene overlapping).
We choose max is better bacause in one of the images there can be cluods, so max value (possibly without coluds) is more informative.

In [296]:
data_sentinel = data_prev.groupby('id').max().reset_index(drop=True).fillna(0)
data_sentinel = data_sentinel.loc[data_cat['id'], sorted(data_sentinel.columns)]

Finally, save the data.

In [298]:
data_sentinel.iloc[:data.shape[0],:].to_csv('data/train_dataset_sentinel.csv', index=False)
data_sentinel.iloc[data.shape[0]:,:].to_csv('data/test_dataset_sentinel.csv', index=False)